# Exercise 4

Welcome to the fourth challenge! The goal of this exercise is to generate a Pub Quiz Crew that can automatically generate pub quiz questions based on random wikipedia articles. Have fun!

## Setup
Let's start by importing the libraries that we will need.

In [ ]:
import os

import wikipedia
from crewai import Agent, Crew, Task, Process
from crewai_tools import BaseTool
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from dotenv import load_dotenv

Before you execute this cell, make sure to provide the environment variables `OPENAI_API_BASE`, `OPENAI_MODEL_NAME`, and `OPENAI_API_KEY` in the `.env` file.

In [ ]:
load_dotenv(override=True)

model_name = os.environ.get("OPENAI_MODEL_NAME", "No model specified in .env file")
print("Using the following LLM model:", model_name)

## Tools

In [ ]:
class RandomTopicGeneratorTool(BaseTool):
    name: str = "Retruns a set of random topics"
    description: str = "Given a number of topics, this tools returns a list of random topics with the specified size"

    def _run(self, num_articles: int) -> list[str]:
        return wikipedia.random(pages=num_articles)

    def cache_function(*args):
        return False


random_topic_generator = RandomTopicGeneratorTool()
wikipedia_search_tool = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

## Agents

In [ ]:
topic_screener = Agent(
    role="Expert Trivia Topic Screener",
    goal=(
        "Look up random topics and select the {num_questions} most interesting ones for a "
        "pub quiz. Consider that at least some of the pub quiz participants should know at "
        "least something about the topics."
    ),
    backstory="You are a famous pub quiz player and you know exactly what trivia topics hold the best questions.",
    tools=[random_topic_generator],
    cache=False,
    verbose=True,
)

topic_researcher = Agent(
    role="Senior Topic Researcher",
    goal=(
        "Given a topic, find out as much as you can about it and write down the "
        "{num_key_infos_per_topic} most important and/or curious facts about the topic."
    ),
    backstory=(
        "After serving 20 years in a OSINT group at the secret service "
        "you decided that it is now time to pursue your passion: applying "
        "your reasearching skills to help a team of pub quiz enthusiasts "
        "to get the best facts about certain topics which they then convert"
        "into unique and compelling questions."
    ),
    tools=[wikipedia_search_tool],
    cache=True,
    verbose=True,
)

quiz_writer = Agent(
    role="World-renowned Quiz Writer",
    goal=(
        "Given a set of {num_questions} topics and facts about these topics, you craft "
        "unique and compelling questions that can be included in a pub quiz. You write "
        "exactly {num_questions} questions, one for each topic, always based on the "
        "facts that are provided to you for that specific topic. You also provide the "
        "correct answer for each question."
    ),
    backstory=(
        "Known by all pub quiz lovers, you excell at writing funny and interesting pub "
        "quiz questions. Many people only go to a pub quiz if you wrote the questions, "
        "and they pay a lot of money for it. Before becoming a famous quiz writer, you "
        "went to pub quizzes yourself for years, until you became bored of the same old "
        "questions. This is why you decided that there was time for a change."
    ),
    cache=False,
    verbose=True,
)

qa_specialist = Agent(
    role="Quiz QA Specialist",
    goal=(
        "Analyze pub quiz questions and point out if there is a question that does not "
        "meet with your quality standards or if a question is inappropriate."
    ),
    backstory=(
        "You've always had a knack for trivia, with a brain wired for absorbing obscure "
        "facts and a passion for perfection. After years of dominating local pub quizzes, "
        "you found yourself in a unique role—Quiz QA Specialist. Your job is to sift "
        "through hundreds of quiz questions daily, ensuring each one is clear, accurate, "
        "and challenging enough to keep participants on their toes. With a red pen in "
        "hand, you meticulously edit or reject any question that doesn't meet your high "
        "standards, knowing that a great quiz is built on the foundation of flawless questions."
    ),
    cache=False,
    verbose=True,
)

ceo = Agent(
    role="CEO of Triviology, the leading pub quiz company",
    goal=(
        "Manage the company and all of its employees to make sure that each employee "
        "works on the tasks that lie in their area of expertise. You orchestrate the "
        "collaboration between different employees to ensure good outcomes. You make "
        "sure that suggestions for improvements are taken seriously and acted upon."
    ),
    backstory=(
        "After climbing the corporate ladder in the competitive world of entertainment, "
        "you became the visionary CEO of Triviology, a leading company in the pub quiz "
        "industry. With a deep love for trivia and a keen business acumen, you've "
        "transformed Triviology from a small startup into a global brand synonymous "
        "with high-quality quiz experiences. Your days are spent strategizing new ways "
        "to innovate, expanding into new markets, and ensuring that every quiz night, "
        "whether in a cozy local pub or a massive online event, delivers both fun and "
        "challenge. Under your leadership, Triviology is not just about asking questions"
        "—it's about creating memorable, engaging experiences for trivia enthusiasts everywhere."
    ),
    cache=False,
    verbose=True,
)

## Tasks

In [ ]:
find_topics = Task(
    description=(
        "Compile a list of {num_random_articles} random topics and select "
        "the {num_questions} most interesting ones that you think could be "
        "a good starting point for creating pub quiz questions."
    ),
    expected_output="A bullet point list with {num_questions} topics",
    agent=topic_screener,
)

research_information = Task(
    description=(
        "Given a topic, find information about the topic and write down "
        "the {num_key_infos_per_topic} most important and/or curious points about the topic."
    ),
    expected_output="A bullet point list with {num_key_infos_per_topic} points per topic",
    agent=topic_researcher,
)

write_questions = Task(
    description=(
        "Given a set of {num_questions} topics and facts about these topics, craft "
        "unique and compelling questions that can be included in a pub quiz, and also "
        "write down the answer to each question. There should be one question for each "
        "topic, and the questions should always be based on the information provided."
    ),
    expected_output="A list with {num_questions} question - answer pairs for a pub quiz.",
    agent=quiz_writer,
    output_file="questions.txt"
)

quality_assesment = Task(
    description=(
        "Analyze pub quiz questions and point out if there is a question that does not "
        "meet with your quality standards or if a question is inappropriate. If youu point "
        "out something that needs to be changed, explicitly state that the Quiz Writer should "
        "have a look at your feedback and implement your suggestions."
    ),
    expected_output=(
        "Either a simple 'questions approved' or a list of specific shortcomings of the "
        "questions. Explain your reasoning in case there are shortcomings."
    ),
    agent=qa_specialist,
)

## Kicking off the Crew

Now it's time to kick off the crew. This part is already implemented for you, so go ahead an execute it. You will be asked for a question, here are some inspirations:

`...`

In [ ]:
crew = Crew(
    agents=[topic_screener, topic_researcher, quiz_writer, qa_specialist],
    tasks=[find_topics, research_information, write_questions, quality_assesment],
    process=Process.hierarchical,
    manager_agent=ceo,
    cache=False,
)

result = crew.kickoff(inputs={"num_questions": 5, "num_random_articles": 250, "num_key_infos_per_topic": 10})

print(result.raw)

In [ ]:
print(crew.usage_metrics)